In [1]:
from collections import Counter

def dic_(allDocument): # alldocument=collection: list
    long_str = " ".join(allDocument)
    b=list(set(list(long_str.split())))
    return sorted(b)

def tf_(doc_que):
    counts = Counter(list(doc_que.split()))
    return dict(counts)

def df_(term, allDocuments):  # df: no. of occurance of a term in whole collection
    dic=dict.fromkeys(term, 0)
    for i in allDocuments:
        n=0
        for word in term:
            if word in i.split():         
                dic[word]=dic[word]+1
            n+=1  
    return dic  
import math

def tfidf_(query_s,c,df,allDocument): # query_s: Series (df['col']), allDocument:list
    
    all_tfidf=[]
    all_tf=[]
    for query in query_s:
        q_tf=tf_(query)
        all_tf.append(q_tf)
        mac=q_tf.values()
        max_tf=max(q_tf.values())
        tfidf=[]
        for word in c:
            if word not in q_tf: # query item does not shown in doc collection
                    value=0
            else:
                value=(1+math.log10(q_tf.get(word)))/(1+math.log10(max_tf))*math.log10(len(allDocument)/df.get(word))
           
            tfidf.append(value)
        all_tfidf.append(tfidf)
    return np.array(all_tfidf), all_tf #vector
import pandas as pd
import numpy as np

df_query=pd.read_excel('df_query_pre.xlsx')
df_doc=pd.read_excel('df_doc_pre.xlsx')

doc_col=df_doc.loc[: , "Text_tok"].tolist() #collection: list

In [2]:
dic_df=pd.read_csv('dictionary.csv')
doc_fre_df=pd.read_csv('document_frequency.csv')
dic=dic_df['0'].tolist()
doc_fre=pd.Series(doc_fre_df.df.values,index=doc_fre_df.term).to_dict()
q_tfidf,q_tf=tfidf_(df_query["Text_tok"],dic,doc_fre,doc_col) # query tf-idf
d_tfidf,d_tf=tfidf_(df_doc["Text_tok"],dic,doc_fre,doc_col) # doc tf-idf

In [3]:
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(a,b):
    cos_sim = dot(a, b)/(norm(a)*norm(b))
    return cos_sim

# tiered index

In [7]:
from pandas import Series, DataFrame
import time
def top_N(K, dataframe, tiers, n, term_key):
# K: the number of top documents
    quantile=1 # split posting list into several same length parts
    for i in range(tiers):
        quantile=quantile-1/tiers
        if quantile<0:
            quantile=0
        quantile_value_list=dataframe.quantile(quantile, axis=1)
        s=dataframe.sub(quantile_value_list,axis=0)
        s=s.where(s>0,0) 
        s=s.where(s<=0,1) 
        columns_sum=s.sum()
        top_doc_list=[i for i in range(len(columns_sum)) if columns_sum[i] >= n*len(term_key)]

        if len(top_doc_list)>=K:
            break
    return top_doc_list

def tiered_index(K, query_tf, documents_tf, tiers, n):
    start = time.time()
    # query_tf are all the query list
    # documents_tf is all the documents list
    a=pd.DataFrame(documents_tf).T # dataframe of all documents
    top_list=[]
    for i in range(len(query_tf)):
        term_key=list(query_tf[i].keys())
        c=pd.DataFrame(a,index=term_key).fillna(value=0)# dataframe of posting list
        List=top_N(K, c, tiers, n, term_key)
        top_list.append(List)
    end = time.time()
    return top_list

# cosine similarity

In [5]:
# cosine similarity
import time
def cosine_similarity_final(q_tfidf,d_tfidf,top_k_list):
    start = time.time()
    sim_all={}
    for q in range(len(q_tfidf)):
        sim={}
        sim_all[q]=sim
        new_d_tfidf=[]
        for index in top_k_list[q]:
            new_d_tfidf.append(d_tfidf[index])# remain tfidf value of return relevant documents

        for d in range(len(new_d_tfidf)):
            sim[d+1]=cosine_similarity(new_d_tfidf[d],q_tfidf[q])

    end = time.time()
    
    
    df_similarity= pd.DataFrame(data=sim_all)
    df_similarity['d_index1']=df_similarity.index
    df1 = df_similarity.reset_index(drop=True)
    df2 = pd.melt(df1, id_vars=["d_index1"], var_name="q_index", value_name="similarity")
    df2=df2.dropna()
    df2['d_index']=sum(top_k_list, [])
    df2 = df2[["d_index","q_index","similarity"]]
    df2=df2.sort_values(by=['q_index','similarity'],ascending=[True,False])
    df_query['q_index']=df_query.index
    df_doc['d_index']=df_doc.index
    df2=pd.merge(df2,df_query[['Query id','q_index']],on='q_index',how='left')
    df2=pd.merge(df2,df_doc[['Doc id','d_index']],on='d_index',how='left')
    df_similarity_final=df2[['Query id', 'Doc id', 'similarity']]
    
    #change label order
    rel_label = pd.read_excel("all2-1-0.qrel.xlsx")
    rel_similarity=pd.merge(df_similarity_final[['Query id','Doc id']],rel_label, on=['Query id','Doc id'], how='left')
    rel_similarity=rel_similarity.fillna(value=0)
    rel_similarity=rel_similarity.drop(['Doc id'],axis=1)
    s = rel_similarity.groupby('Query id')['Rel_level'].apply(lambda x: x.tolist())
    correct_input=s.values
    return correct_input, end-start



# MAP

In [6]:
#MAP
import numpy as np 
# presicion 
def precision(y_true):
#y_true: this list reordered y_true list
    p=[]
    a=0
    for i in range(len(y_true)):
        if y_true[i]==1:
            a+=1
            precision=a/(i+1)
            p.append(precision)
    #print(p)
    return p

# average precision
def AP(y_true):
    p=precision(y_true)
    if len(p)!=0:
        AP=sum(p)/len(p)
    else:
        AP=0
    #print(AP)
    return AP


def MAP(list_true):# query list 
    ap=[]
    for i in range(len(list_true)):
        ap.append(AP(list_true[i]))
    total=sum(ap)
    #print(ap)
    return total/(len(list_true))
#print(MAP(correct_input))

# NDCG

In [11]:
#NDCG
import numpy as np 
def ndcg(correct):
    dcg=0
    for i in range(len(correct)):
        gain=2**correct[i]-1
        discounts=np.log2(i+2)
        dcg=dcg+gain/discounts
    
    #idcg
    order = np.argsort(correct)
    sort_correct = np.take(correct, order[::-1])
    idcg=0
    for i in range(len(sort_correct)):
        gain=2**sort_correct[i]-1
        discounts=np.log2(i+2)
        idcg=idcg+gain/discounts
    #print(dcg)
    #print(idcg)
    if idcg==0:
        c=0
    else:
        c=dcg/idcg
    return c

def mean_ndcg(correct_list):
    b=0
    for correct in correct_list:
        a=ndcg(correct)
        b=b+a
    return b/len(correct_list)

# whole process

In [12]:
def whole_process(K,q_tf,d_tf,tiers,q_tfidf,d_tfidf,precision_list):
    time=[]
    MAP_value=[]
    ndcg_value=[]

    
    for value in precision_list:
        top_k_list=tiered_index(K, q_tf,d_tf, tiers, value)

        correct_input,time_similarity=cosine_similarity_final(q_tfidf, d_tfidf, top_k_list)
        time.append(time_similarity)
        MAP_value.append(MAP(correct_input))
        ndcg_value.append(mean_ndcg(correct_input))
        
    frame=pd.DataFrame(data=[time,MAP_value,ndcg_value],columns=precision_list,index=['time','MAP','NDCG'])
    return frame
        

In [13]:
# Example
tiers=3
precision_list=[0.06]
K=50
a=whole_process(K,q_tf,d_tf,tiers,q_tfidf,d_tfidf,precision_list)

In [14]:
print(a)

            0.06
time  130.249000
MAP     0.157727
NDCG    0.461134


In [ ]:
# For testing:
# There are three varibles: 
# 1.Number of tier
# 2.Required percentage of query terms a document contains 
#   This a percentage value list, user can put diferent value in it and see their 
#   corresponding result
# 3.Top K value

#-----------------------------------------------
# This is the code for testing:
#-----------------------------------------------
#tiers= 
#precision_list=[ ]
#K=
#a=whole_process(K,q_tf,d_tf,tiers,q_tfidf,d_tfidf,precision_list)
#print(a)